 ## Techniques to reduce runtime in Pandas

### Key Takeaways
##### 1. Use Vectorisation 
##### 2. When converting a column to datetime, specify the format
##### 3. The built in function .iterrows is quicker than for loops
##### 4. Use .eval expressions on very large datasets
##### 5. When combining multiple dataframes, use pd.concat. But only use it once, on a list of all the dataframes    
        5i. If the original data is in dictionary form, it is much quicker to create the combined dataframe as the final step on a list of dictionaries

In [1]:
#Import modules
import pandas as pd
import numpy as np
rng = np.random.RandomState(123)
#Import data
date_data = pd.read_csv('data/transactions.csv')
time_data = pd.read_csv('data/temperature_and_wind.csv',index_col=0)


In [2]:
#Create Timing decorator
import functools
import gc
import itertools
import sys
from timeit import default_timer as _timer


def timeit(_func=None, *, repeat=3, number=1000, file=sys.stdout):
    """Decorator: prints time from best of `repeat` trials.
    Mimics `timeit.repeat()`, but avg. time is printed.
    Returns function result and prints time.
    You can decorate with or without parentheses, as in
    Python's @dataclass class decorator.
    kwargs are passed to `print()`.
    >>> @timeit
    ... def f():
    ...     return "-".join(str(n) for n in range(100))
    ...
    >>> @timeit(number=100000)
    ... def g():
    ...     return "-".join(str(n) for n in range(10))
    ...
    """

    _repeat = functools.partial(itertools.repeat, None)

    def wrap(func):
        @functools.wraps(func)
        def _timeit(*args, **kwargs):
            # Temporarily turn off garbage collection during the timing.
            # Makes independent timings more comparable.
            # If it was originally enabled, switch it back on afterwards.
            gcold = gc.isenabled()
            gc.disable()

            try:
                # Outer loop - the number of repeats.
                trials = []
                for _ in _repeat(repeat):
                    # Inner loop - the number of calls within each repeat.
                    total = 0
                    for _ in _repeat(number):
                        start = _timer()
                        result = func(*args, **kwargs)
                        end = _timer()
                        total += end - start
                    trials.append(total)

                # We want the *average time* from the *best* trial.
                # For more on this methodology, see the docs for
                # Python's `timeit` module.
                #
                # "In a typical case, the lowest value gives a lower bound
                # for how fast your machine can run the given code snippet;
                # higher values in the result vector are typically not
                # caused by variability in Python’s speed, but by other
                # processes interfering with your timing accuracy."
                best = min(trials) / number
                print(
                    "Best of {} trials with {} function"
                    " calls per trial:".format(repeat, number)
                )
                print(
                    "Function `{}` ran in average"
                    " of {:0.3f} seconds.".format(func.__name__, best),
                    end="\n\n",
                    file=file,
                )
            finally:
                if gcold:
                    gc.enable()
            # Result is returned *only once*
            return result

        return _timeit

    # Syntax trick from Python @dataclass
    if _func is None:
        return wrap
    else:
        return wrap(_func)

### 1. Vectorization is much quicker than .apply function
    Vectorisation is the ability to run an operation across a whole Series, Index, or even DataFrame at once

In [3]:
df = pd.DataFrame(np.random.randint(0, 11, size=(1000000, 5)), columns=('a','b','c','d','e'))

In [4]:
%%timeit
#Apply function
def func(a,b,c,d,e):
    if e == 10:
        return c*d
    elif (e < 10) and (e>=5):
        return c+d
    elif e < 5:
        return a+b
df['new'] = df.apply(lambda x: func(x['a'], x['b'], x['c'], x['d'], x['e']), axis=1)

16.8 s ± 2.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
#Vectorization
df['new'] = df['c'] * df['d'] #default case e = =10
mask = df['e'] < 10
df.loc[mask,'new'] = df['c'] + df['d']
mask = df['e'] < 5
df.loc[mask,'new'] = df['a'] + df['b']

68.4 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. Specifying the format of datetime when converting

    Pandas will take a significant amount of runtime to interpret the data without formatting provided. Note this is the case for Datetime conversion, date conversion is not affected

##### DateTime

In [6]:
df = time_data[:10000].copy()

@timeit(repeat=3, number=10)
def convert(df, column_name):
    return pd.to_datetime(df[column_name])
    
df['Date Time'] = convert(df, 'Date Time')

Best of 3 trials with 10 function calls per trial:
Function `convert` ran in average of 0.619 seconds.



In [7]:
df = time_data[:10000].copy()

@timeit(repeat=3, number=10)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

df['Date Time'] = convert_with_format(df, 'Date Time')

Best of 3 trials with 10 function calls per trial:
Function `convert_with_format` ran in average of 0.022 seconds.



##### Date

In [8]:
df = date_data.copy()

@timeit(repeat=3, number=10)
def convert(df, column_name):
    return pd.to_datetime(df[column_name])
    
df['date'] = convert(df, 'date')

Best of 3 trials with 10 function calls per trial:
Function `convert` ran in average of 0.422 seconds.



In [9]:
df = date_data.copy()

@timeit(repeat=3, number=10)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%Y-%m-%d')

df['date'] = convert_with_format(df, 'date')

Best of 3 trials with 10 function calls per trial:
Function `convert_with_format` ran in average of 0.423 seconds.



### 3. Use .iterrows() or .apply() instead of looping through range

In [10]:
# Set Up functions (IGNORE)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

def times_of_day(hour,temp):
    if hour < 6:
        rate = 3
    elif hour < 12:
        rate = 2
    elif hour < 16:
        rate = 1
    elif hour <20:
        rate = 2
    else:
        rate = 3
    return rate*temp


##### For Loop

In [11]:
# Looping through range
df = time_data[:100000].copy()


@timeit(repeat=3, number=10)
def get_temp_perhour(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')
    temp_hour_list = []
    for i in range(len(df)):                        ##### Looping through range
        hour = df.iloc[i]['Date Time'].hour
        temp = df.iloc[i]['T (degC)']
        temp_hour_list.append(times_of_day(hour,temp))

    df['temp-hour'] = temp_hour_list
get_temp_perhour(df)

Best of 3 trials with 10 function calls per trial:
Function `get_temp_perhour` ran in average of 13.660 seconds.



#### iterrows

In [12]:
# Using iterrows
df = time_data[:100000].copy()

def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

@timeit(repeat=3, number=10)
def get_temp_perhour(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')
    temp_hour_list = []
    for index, row in df.iterrows():                ###### Using iterrows
        hour = row['Date Time'].hour
        temp = row['T (degC)']
        temp_hour_list.append(times_of_day(hour,temp))
    df['temp-hour'] = temp_hour_list

get_temp_perhour(df)

Best of 3 trials with 10 function calls per trial:
Function `get_temp_perhour` ran in average of 2.627 seconds.



#### The apply function is quicker than both of the above

In [13]:
#Using apply function
df = time_data[:100000].copy()

def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d.%m.%Y %H:%M:%S')

@timeit(repeat=3, number=10)
def apply_get_temphour(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')
    df['temp-hour'] = df.apply(                                            #Using apply function
        lambda row: times_of_day(row['Date Time'].hour,row['T (degC)']),
        axis=1
    )
apply_get_temphour(df)

Best of 3 trials with 10 function calls per trial:
Function `apply_get_temphour` ran in average of 0.906 seconds.



#### Vectorisation is still quicker than all of the above
##### It won't use the original function but vectorisation can be coded in multiple ways

    Here .isin() is used though mask can be also used like in case 1

In [14]:
df = time_data[:100000].copy()

@timeit(repeat=3, number=10)
def use_isin(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')

    time = df['Date Time'].dt.hour
    early = time.isin(range(0,6))
    morning = time.isin(range(6,12))
    afternoon = time.isin(range(12,16))
    evening = time.isin(range(16,20))
    late = time.isin(range(20,25))


    df.loc[early, 'temp-hour'] = df.loc[early, 'T (degC)'] * 3
    df.loc[morning, 'temp-hour'] = df.loc[morning, 'T (degC)'] * 2
    df.loc[afternoon, 'temp-hour'] = df.loc[afternoon, 'T (degC)'] * 1
    df.loc[evening, 'temp-hour'] = df.loc[evening, 'T (degC)'] * 2
    df.loc[late, 'temp-hour'] = df.loc[late, 'T (degC)'] * 3

use_isin(df)

Best of 3 trials with 10 function calls per trial:
Function `use_isin` ran in average of 0.016 seconds.



    The built in pd.cut can be used instead of .isin for less lines of code and slight runtime improvement

In [15]:
df = time_data[:100000].copy()

@timeit(repeat=3, number=10)
def use_cut(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')

    hour_factor = pd.cut(x=df['Date Time'].dt.hour,
                        bins=[0,5,11,15,19,24],
                        include_lowest=True,
                        labels = [3,2,1,4,5]).astype(int)
    
    hour_factor = hour_factor.replace({4:2,5:3})
    
    df['temp-hour'] =  hour_factor * df['T (degC)']

use_cut(df)

Best of 3 trials with 10 function calls per trial:
Function `use_cut` ran in average of 0.012 seconds.



    Numpy's digitize function works in the same way as pd.cut, but it's slightly quicker

In [16]:
df = time_data[:100000].copy()

@timeit(repeat=3, number=10)
def use_digitize(df):
    df['Date Time'] = convert_with_format(df, 'Date Time')

    bins = np.digitize(df['Date Time'].dt.hour.values,
                        bins=[6,12,16,20,24])
    factor = np.array([3,2,1,2,3])
    
    df['temp-hour'] =  factor[bins]

use_digitize(df)

Best of 3 trials with 10 function calls per trial:
Function `use_digitize` ran in average of 0.009 seconds.



### 4. Use eval() for large dataframes

    Why it is quicker for large dataframes: https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html

##### Eval uses string expressions to efficiently compute operations. In the case below it's over 50% faster

In [17]:
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4,df5 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(5))

In [18]:
%timeit (-df1 * df2 / (df3 + df4) - df5)

163 ms ± 4.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit pd.eval('-df1 * df2 / (df3 + df4) - df5')

45.6 ms ± 589 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 5. What is the fastest way to concat multiple dataframes?

In [20]:
def generate_df(i):
    df = pd.DataFrame(i,index=[0],columns=['x', 'y'])
    return df

def generate_dictionary(i):
    return {'x':i,'y':i}

#### Concat each dataframe in a for loop (Slowest)

In [21]:
@timeit
def concat_each_df(): 
    combined = pd.DataFrame()

    for i in range(1,1000):
        df = generate_df(i)
        combined = pd.concat([combined, df]) #Concat each new df on current df
    return combined.reset_index()
    
concat_each_df()

Best of 3 trials with 1000 function calls per trial:
Function `concat_each_df` ran in average of 0.259 seconds.



,index,x,y
0,0,1,1
1,0,2,2
2,0,3,3
3,0,4,4
4,0,5,5
...,...,...,...
994,0,995,995
995,0,996,996
996,0,997,997
997,0,998,998


#### Use concat only once on a list of all dataframes

In [22]:
@timeit
def list_of_dfs(): 
    dfs = []
    for i in range(1,1000):
        df = generate_df(i)
        dfs.append(df)                         ## Add new df to list
    return pd.concat(dfs).reset_index(drop=True) ## Apply concat function to list of dfs

list_of_dfs()

Best of 3 trials with 1000 function calls per trial:
Function `list_of_dfs` ran in average of 0.145 seconds.



,x,y
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
994,995,995
995,996,996
996,997,997
997,998,998


#### Keep data in dictionary format. Create a final dataframe from list of dictionaries

In [23]:
@timeit
def df_from_dict():
    rows = []
    for i in range(1, 1000):
    # Instead of generating a dataframe, generate a dictionary
        dictionary = generate_dictionary(i)
        rows.append(dictionary)

    return pd.DataFrame(rows)

df_from_dict()

Best of 3 trials with 1000 function calls per trial:
Function `df_from_dict` ran in average of 0.001 seconds.



,x,y
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
994,995,995
995,996,996
996,997,997
997,998,998
